In [1]:
import numpy as np
import random
import math

In [2]:
def line_generate(lenght, m):
    """
    Creating a line with m random segments
    """
    line = np.zeros(lenght,)
    ab = np.array(sorted(random.sample(range(0, lenght-1), m*2))) # random start and end of the segments
    ab = ab.reshape((m,2)) # start and end of the segments coordinates
    # positive points coordinates
    ones = np.arange(ab[0,0],ab[0,1])
    for i in range(1,m):
        ones = np.concatenate((ones, np.arange(ab[i,0],ab[i,1])))
    line[ones] = 1  # put the generated segments on the line
    return line, ones, ab

In [3]:
def segment_number(c, ab):
    """
    For a point c find a segment it belongs to
    """
    for i in range(0,ab.shape[0]):
        if (c >= ab[i,0]) & (c <= ab[i,1]):
            return i  

In [4]:
def find_associated(c, ab, base, L):
    """
    For a given point c find its associated point 𝑐′
    """
    # distance from the farthest point to the base
    b = math.sqrt(c**2+base[1]**2)
    # distance from the farthest point to greedy
    dist = (L**2-2*L*b)/(2*(L-b-c))
    c_ass = c-math.floor(dist)
    for i in range(0,ab.shape[0]):
        if (c_ass > ab[i-1,1]) & (c_ass < ab[i,0]):
            c_ass = ab[i,0]
        if (c_ass < ab[0,0]):
            c_ass = ab[0,0]  
    return c_ass

In [5]:
def candidates(base, L, ab):
    """
    Making the greedy jumps to find candidate points c
    """
    on_gap = False
    xB = ab[ab.shape[0]-1,1]
    c = np.array(xB)
    while not on_gap:
        # distance from the farthest point to the base
        b = math.sqrt(xB**2+base[1]**2)
        # distance from the farthest point to greedy
        dist = (L**2-2*L*b)/(2*(L-b-xB))
        # greedy point coordinate
        xA = xB-math.floor(dist)
  #      print("xB", xB)
  #      print("xA", xA)
        c = np.append(c, xA)
        
        xB = xA
        for i in range(0,ab.shape[0]):
            if (xA > ab[i-1,1]) & (xA < ab[i,0]):
                on_gap = True 
            if (xA < ab[0,0]):
                on_gap = True 
    c = c[:c.size-1]
    return c

In [6]:
def all_candidates(base, L, ab):
    """
    Construction of the candidate set 𝐶={c_i}
    """    
    ab_cut = ab
    c = np.empty(0)
    while not (ab_cut.size == 0):
        c_i = candidates(base, L, ab_cut)
        c = np.append(c, c_i)       
        ab_cut = ab_cut[:ab_cut.shape[0]-1]
    return c

In [7]:
def perimeter(point1, point2, base):
    return max(point1, point2) - min(point1, point2) + math.sqrt(point1**2+base[1]**2) + math.sqrt(point2**2+base[1]**2)

In [8]:
def separate_left_right(ab, base):
    """
    Separate the segment into two groups: left and right from the base
    """

    if (ab[-1,0]>=0):
        ab_right = ab[np.argmax(ab[:,0] > 0):]
    else:
        ab_right = np.empty((0,2))
        
    if (ab[0,1]<=0):
        ab_left = -np.flip(ab[:np.argmax(ab[:,1] > 0)])
    else:
        ab_left = np.empty((0,2))       
    
    
    for i in range(0,ab.shape[0]):
        if (0 > ab[i,0]) & (0 < ab[i,1]):
            central_right = np.array([[0, ab[i,1]]])
            central_left = -(np.array([[0,ab[i,0]]]))
            ab_right = np.append(central_right, ab_right, axis=0)
            ab_left = np.append(central_left, ab_left, axis=0)
    return ab_left, ab_right

In [9]:
def find_central_tours(c_left, c_right , base, L):
    """
    Find a set of segments that can be covered from left to right and vice versa
    """   
    j = 0
    central_tours = np.empty((0,3))
    tour_lenght = 0
    while (tour_lenght <= L) and (j < c_left.size):
        central_tours_1 = np.empty((0,3))
        i = 0
        while (tour_lenght <= L) and (i < c_right.size):
            dist_left = math.sqrt(c_left[j]**2+base[1]**2)
            dist_right = math.sqrt(c_right[i]**2+base[1]**2)
            tour_lenght = c_left[j] + c_right[i] + dist_left + dist_right
            tour = np.array((c_left[j], c_right[i], tour_lenght)).reshape(1,3)
            central_tours_1 = np.append(central_tours_1, tour, axis=0)
            i += 1
        central_tours = np.append(central_tours, central_tours_1[central_tours_1[:,2]<=L,:], axis=0)
        tour_lenght = 0
        j += 1
    return central_tours

In [10]:
def DP_one_side(c, ab, base, L):   

    """
    One side dynamic programming implementation
    """
    c = np.sort(c)  
    E = np.zeros(c.size) # check if zeros are ok
    Start_Points = - np.ones((c.size,c.size)) # check if zeros are ok
    End_Points = - np.ones((c.size,c.size))

    for k in range(0, c.size):
        c_ass = find_associated(c[k], ab, base, L)
        
        j = np.arange(segment_number(c_ass, ab), segment_number(c[k], ab)+1)  
        
        if c_ass == ab[0,0]:
            E[k] = perimeter(c[k], ab[0,0], base)  

            
            Start_Points[k,0] = c[k]
            End_Points[k,0] = ab[0,0]
            
        elif c_ass in ab[1:,0]:      
            
            func = np.empty(0)
            for i in j:
                func = np.append(func, perimeter(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])
            E[k] = min(func)
            index_min = np.argmin(func)+j[0]
            
            Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
            End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                                   
            Start_Points[k, np.argmax(Start_Points[k,:]<0)] = c[k]
            End_Points[k, np.argmax(End_Points[k,:]<0)] = ab[index_min,0]
    
        else:             
            func = np.empty(0)
            if (c_ass < ab[segment_number(c[k], ab),0]): 
                for i in j[1:]:
                    func = np.append(func, perimeter(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])  
            else:
                func = [1000000.0]
            func2 = L + E[c.tolist().index(c_ass)]
            
            if (func2>min(func)):
                index_min = np.argmin(func)+j[0]+1
               
                Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,np.argmax(End_Points[k,:]<0)] = ab[index_min,0]
            else:
                Start_Points[k,:] = Start_Points[c.tolist().index(c_ass),:] 
                #check
                End_Points[k,:] = End_Points[c.tolist().index(c_ass),:] 
                End_Points[k,np.argmax(End_Points[k,:]<0)] = c_ass
                
            Start_Points[k,np.argmax(Start_Points[k,:]<0)] = c[k]                    
            E[k] = min(func2, min(func))
            
    Start_Points = Start_Points[c.size-1]
    End_Points = End_Points[c.size-1]
    return E[c.size-1], Start_Points[Start_Points>=0], End_Points[End_Points>=0]

In [11]:
def all_left_right_DP(c, c_first, ab, base, L):
    """
    Go through all the options and do one side DP removing the central segment
    """
    E = np.zeros(c_first.size)
    StartPoints = np.zeros((c_first.size,c.size))
    EndPoints = np.zeros((c_first.size,c.size))
    ab_i = ab.astype(np.float64)
    for i in range(0, c_first.size):
        if c_first[i] != ab_i[0,1]:
            ab_i[0,0] = c_first[i]
        else:
            ab_i = ab[ab[:,0]>c_first[i]].astype(np.float64)
        if (c[c>c_first[i]].size != 0):
            E[i],  Start_Points, End_Points = DP_one_side(c[c>c_first[i]], ab_i, base, L) 
            StartPoints[i,:Start_Points.size] = Start_Points
            EndPoints[i,:End_Points.size] = End_Points
            
        else: E[i] = 0
    return E, StartPoints, EndPoints

In [12]:
def join_tours(StartPoints, EndPoints):
    """
    Save right and left points of the tours together in one array
    """  
    Start = np.array([StartPoints[StartPoints>=0]])
    End = np.array([EndPoints[EndPoints>=0]])
    Tour = np.append(Start, End, axis = 0)
    idx = np.argwhere(np.all(Tour[..., :] == 0, axis=0))
    
    return np.delete(Tour, idx, axis=1)

In [13]:
def DP_both_sides(ab, base, L):
    """
    Full DP solution with all comparisons near the projection point
    """  
    
    ab_left, ab_right = separate_left_right(ab, base)
    
    c_right = np.sort(all_candidates(base, L, ab_right))
    c_left =  np.sort(all_candidates(base, L, ab_left))
    
    
    central_tours = find_central_tours(c_left, c_right , base, L)
    
    
    if not (central_tours.size == 0):  
        c_left_first = np.unique(np.sort(central_tours[:,0]))
        c_right_first = np.unique(np.sort(central_tours[:,1]))   
        
        
        E_left, StartPoints_left, EndPoints_left = all_left_right_DP(c_left, c_left_first, ab_left, base, L)
        E_right, StartPoints_right, EndPoints_right = all_left_right_DP(c_right, c_right_first, ab_right, base, L)
        
        
        left_central = np.append(c_left_first.reshape(E_left.shape[0],1), E_left.reshape(E_left.shape[0],1), axis=1)
        right_central = np.append(c_right_first.reshape(E_right.shape[0],1), E_right.reshape(E_right.shape[0],1), axis=1)
      
        
      
        totalL = np.zeros(central_tours.shape[0])
        for i in range (0, central_tours.shape[0]):
            a = tuple(np.argwhere(left_central == central_tours[i,0])[0])
            b = tuple(np.argwhere(right_central == central_tours[i,1])[0])
            totalL[i] = left_central[a[0],1] + right_central[b[0],1] + float(central_tours[i,2])
        
        c = tuple(np.argwhere(left_central == central_tours[np.argmin(totalL),0])[0])
        d = tuple(np.argwhere(right_central == central_tours[np.argmin(totalL),1])[0])
    else: 
        totalL = np.array([100000000])

    E_left_max, StartPoints_left_max, EndPoints_left_max = DP_one_side(c_left, ab_left, base, L)
    E_right_max, StartPoints_right_max, EndPoints_right_max = DP_one_side(c_right, ab_right, base, L)
    
    
    if (E_left_max + E_right_max) <= min(totalL):
        Tour_right = join_tours(StartPoints_right_max, EndPoints_right_max)
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = -join_tours(StartPoints_left_max, EndPoints_left_max)


        Tour = np.append(np.flip(Tour_left,1), Tour_right, axis = 1)
        
    else:  
        Tour_right = join_tours(StartPoints_right[d[0]], EndPoints_right[d[0]])
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = -join_tours(StartPoints_left[c[0]], EndPoints_left[c[0]])
        Tour_central = np.array([[-central_tours[np.argmin(totalL),0], central_tours[np.argmin(totalL),1] ]]).reshape((2, 1))
        
        Tour = np.append(np.flip(Tour_left,1), Tour_central, axis = 1)
        Tour = np.append(Tour, Tour_right, axis = 1)
        
    TotalLenght = min((E_left_max + E_right_max), min(totalL)) 
    TotalLenght_calc = 0
    for i in range(Tour.shape[1]):
        print('Tour', i+1, 'start:', Tour[0,i])
        print('Tour', i+1, 'end:', Tour[1,i])
        print('Tour', i+1, 'length:', perimeter(Tour[0,i], Tour[1,i], base))
        TotalLenght_calc+=perimeter(Tour[0,i], Tour[1,i], base)
    print('Total number of tours:', Tour.shape[1])
    
    print('Total length:', TotalLenght)
    print('Check:', TotalLenght_calc==TotalLenght)
    
    return Tour, TotalLenght  

In [14]:
def distance(point, y_base):
    """
    Distances to the base station
    """
    dist = math.sqrt(point**2+y_base**2)
    return dist 

In [15]:
def tour_lenght(tour, y_base):
    dist_left = distance(tour[0],y_base)
    dist_right = distance(tour[1],y_base)
    lenght = abs(tour[0]-tour[1]) + dist_left + dist_right
    return lenght

In [16]:
def total_lenght(t_set, y_base):
    sum=0
    for t in t_set:
        sum += tour_lenght(t, y_base)
    return sum   

In [17]:
def to_the_min_disribution(Tour, y_base):
    m = Tour.shape[0]-1
    
    Tour1 = np.empty(0)
    Tour2 = np.empty(0)
    l1 = 0
    l2 = 0
    
    
    while Tour.size>1:
        #Find the farthest tour from the base:
        if distance(Tour[0,0],base[1]) >= distance(Tour[m,1],y_base):
            farthest_tour = Tour[0]
            Tour = Tour[1:]
        else:
            farthest_tour = Tour[m]
            Tour = Tour[:m]
        m -= 1
        
        #Add the farthest tour to the smallest set
        if l1 < l2:
            Tour1 = np.append(Tour1, farthest_tour)
            l1 += tour_lenght(farthest_tour, y_base)
        else:
            Tour2 = np.append(Tour2, farthest_tour)
            l2 += tour_lenght(farthest_tour, y_base)
        
    Tour1 = Tour1.reshape(Tour1.shape[0]//2,2)
    Tour2 = Tour2.reshape(Tour2.shape[0]//2,2)

    # Make Tour2 always be the biggest
    if l1 > l2:
        Tour1, Tour2 = Tour2, Tour1
        l1, l2 = l2, l1

    print("T1 =", Tour1)
    print("T1 total length: l1 =", l1)
    print("T2 =",Tour2)
    print("T2 total length: l2 =", l2)   
    return Tour1, Tour2, l1, l2    

In [18]:
def big_bro(t):
    # Find a tour tour just one point bigger than a given
    if (t[0]>0) and (t[1]>0): 
        return [t[0]-1, t[1]]
    if (t[0]<0) and (t[1]<0): 
        return [t[0], t[1]+1]
    if (t[0]<0) and (t[1]>0): 
        if abs(t[0])>abs(t[1]):
            return [t[0], t[1]+1]
        else:
            return [t[0]-1, t[1]]

In [19]:
def candidates_to_enlarge(T1, T2, y_base):
    T_en_left = np.empty(0)
    T_en_right = np.empty(0)
    for t1 in T1:
        for t2 in T2:
            if (t1[0]<0) and (t1[0]==t2[1]) and (tour_lenght(big_bro(t2), y_base)>L) and (tour_lenght(big_bro(t1), y_base)<L):
                T_en_left = np.append(T_en_left, t1)
                T_en_left = T_en_left.reshape(T_en_left.shape[0]//2,2)
            if (t1[1]>0) and (t1[1]==t2[0]) and (tour_lenght(big_bro(t2), y_base)>L) and (tour_lenght(big_bro(t1), y_base)<L):
                T_en_right = np.append(T_en_right, t1)
                T_en_right = T_en_right.reshape(T_en_right.shape[0]//2,2)
                
    return T_en_left, T_en_right

In [20]:
def cutting_improvement(t, l, y_base):
    c = distance(t[0], y_base)
    f = distance(t[1], y_base)
    s = t[1]-t[0]

    #Option 1: Left tour for T1
    x1 = (l-c*2)/2
    d1 = distance(t[0]+x1, y_base)
    r1 = d1 - c - x1

    #Option 2: Left tour for T2
    x2 = (s*2+f*2-l)/2
    d2 = distance(t[0]+x2, y_base)
    r2 = d2 - s - f + x2

    if r2<r1:
        return x2,r2,2
    else:
        return x1,r1,1

In [21]:
def enlarging_improvement(farthest, closest, l, L, y_base):
    x = l/2
    f = distance(farthest, y_base)
    d = distance(farthest+x, y_base)
    c = distance(closest, y_base)
    
    if (farthest+x-closest+c+d > L):
        dist = 0.5*(L**2-2*L*c)/(closest+L-c)
        x=dist+closest-farthest
        d = distance(farthest+x, y_base)
    s = d-f-x
    
    return x,s

In [22]:
# Step 0: Generate a random line
line, ones, ab = line_generate(400, 3)
base = np.array((200,20000)) # base station coordinates
L =  2*math.sqrt((max(base[0], 400-base[0]))**2+base[1]**2)+15

In [23]:
L

40016.9999500025

In [24]:
ab

array([[181, 223],
       [225, 265],
       [314, 336]])

In [25]:
# Step 1: MinSum solution with DP
Tour, TotalLenght = DP_both_sides(ab-base[0], base, L)
Tour = np.transpose(Tour)

Tour 1 start: -19.0
Tour 1 end: -3.0
Tour 1 length: 40016.009249997966
Tour 2 start: -3.0
Tour 2 end: 7.0
Tour 2 length: 40010.00144999997
Tour 3 start: 7.0
Tour 3 end: 23.0
Tour 3 length: 40016.01444999559
Tour 4 start: 25.0
Tour 4 end: 33.0
Tour 4 length: 40008.04284997536
Tour 5 start: 33.0
Tour 5 end: 49.0
Tour 5 length: 40016.087249891396
Tour 6 start: 49.0
Tour 6 end: 65.0
Tour 6 length: 40016.16564963102
Tour 7 start: 114.0
Tour 7 end: 120.0
Tour 7 length: 40006.6848941211
Tour 8 start: 120.0
Tour 8 end: 136.0
Tour 8 length: 40016.82239141484
Total number of tours: 8
Total length: 320107.7527440975
Check: False


In [26]:
# Step 2: Distribute T between two sets T1  y T2  such as l(T2)>l(T1)
Tour1, Tour2, l1, l2 = to_the_min_disribution(Tour, base[1])

T1 = [[114. 120.]
 [ 49.  65.]
 [ 25.  33.]
 [-19.  -3.]]
T1 total length: l1 = 160046.90264372545
T2 = [[120. 136.]
 [ 33.  49.]
 [  7.  23.]
 [ -3.   7.]]
T2 total length: l2 = 160058.92554130178


In [27]:
# Step 3: Get the sets of candidates to enlarge:
T_enlarge_left, T_enlarge_right = candidates_to_enlarge(Tour1, Tour2, base[1])

In [28]:
T_enlarge_right

array([[114., 120.],
       [ 25.,  33.]])

In [29]:
# Step 4: Improve until l(T2)=l(T1)
l= l2-l1
while l>0:
    Table = np.empty(0)
    T_cut = np.empty(0)
    # Get the set of candidates to cut:
    for t2 in Tour2:
        if (distance(t2[1], base[1]) + distance(t2[0], base[1]))<l:
            T_cut = np.append(T_cut, t2)
            T_cut = T_cut.reshape(T_cut.shape[0]//2,2)
            
    # Find the improvement after each possible cutting
    for t in T_cut:
        x_cut, d_cut, idx = cutting_improvement(t,l, base[1])
        Table = np.append(Table, np.array([t[0],t[1], x_cut, d_cut, idx]))
    
    # Find the improvement after each possible enlarging
    for t in T_enlarge_right:
        farthest = abs(t[1])
        closest = abs(t[0])
        x_enlarge_right, d_enlarge_right  = enlarging_improvement(farthest, closest,l, L, base[1])
        Table = np.append(Table, np.array([t[0],t[1], x_enlarge_right, d_enlarge_right, 3]))
        
    for t in T_enlarge_left:
        farthest = abs(t[0])
        closest = abs(t[1])
        x_enlarge_left, d_enlarge_left  = enlarging_improvement(farthest, closest,l,L, base[1])
        Table = np.append(Table, np.array([t[0],t[1], x_enlarge_left, d_enlarge_left, 4]))
        
    print(Table)
    if Table.size == 0:
        break
    Table = Table.reshape(Table.shape[0]//5,5)
    
    
    for tab in Table:
        if tab[3]==min(Table[:,3]):
            winner = tab
    
    win_tour = np.transpose([winner[0],winner[1]])
    
    # Apply cutting
    if winner[4]==1:
        for t2 in Tour2:
            if t2[0] == win_tour[0]:
                start = t2[0]
                t2[0] = t2[0]+np.round(winner[2])
                Tour1 = np.append(Tour1, [start, t2[0]])
                Tour1 = Tour1.reshape(Tour1.shape[0]//2,2)

    if winner[4]==2:
        for t2 in Tour2:
            if t2[0] == win_tour[0]:
                end = t2[1]
                t2[1] = t2[0]+np.round(winner[2])
                Tour1 = np.append(Tour1, [t2[1], end])
                Tour1 = Tour1.reshape(Tour1.shape[0]//2,2)    

    # Apply enlarging
    if winner[4]==3:
        for t1 in Tour1:
            for t2 in Tour2:
                if (t1[0]==winner[0]) and (t1[1]==t2[0]):
                    t1[1] = t1[1]+np.floor(winner[2])
                    t2[0] = t2[0]+np.floor(winner[2])
                    # Remove t1 from the set T_enlarge_right
                    for i in range(T_enlarge_right.shape[0]):
                        if T_enlarge_right[i,0]==t1[0]:
                            T_enlarge_right = np.delete(T_enlarge_right, i,0)
                     
    if winner[4]==4:
        for t1 in Tour1:
            for t2 in Tour2:
                 if (t1[0]==winner[0]) and (t1[0]==t2[1]):
                     t1[0] = t1[0]-np.floor(winner[2])
                     t2[1] = t2[1]-np.floor(winner[2])
                     # Remove t1 from the set T_enlarge_left
                     for i in range(T_enlarge_left.shape[0]):
                         if T_enlarge_left[i,1]==t1[1]:
                             T_enlarge_left = np.delete(T_enlarge_left, i,0)
                
    l1 = total_lenght(Tour1, base[1])
    l2 = total_lenght(Tour2, base[1])
    l= l2-l1
    print("_____________________")

[114.         120.           6.01144879  -5.97447736   3.
  25.          33.           6.01144879  -6.00062648   3.        ]
_____________________
[ 1.14000000e+02  1.20000000e+02  1.14487882e-02 -1.13800934e-02
  3.00000000e+00]
_____________________
[]


In [30]:
Table

array([], dtype=float64)

In [31]:
l

0.022897576389368623

In [32]:
Tour2

array([[120., 136.],
       [ 39.,  49.],
       [  7.,  23.],
       [ -3.,   7.]])

In [33]:
Tour1

array([[114., 120.],
       [ 49.,  65.],
       [ 25.,  39.],
       [-19.,  -3.]])

In [34]:
l2

160052.9363412842

In [35]:
l1 = total_lenght(Tour1, base[1])
l2 = total_lenght(Tour2, base[1])

In [36]:
T_enlarge_left

array([], dtype=float64)

In [37]:
np.delete(T_enlarge_left, 2, 0)

IndexError: index 2 is out of bounds for axis 0 with size 0